In [1]:
from IPython.core.debugger import Tracer

In [2]:
g = 2
n = 4
indexes = [str(i) for i in range(1, n+1)]

We first create the ring of function on the strongly non-special locus. For now, let's stick to the genus two case, so we don't need the $c_{ij}^k$.

In [119]:
def one_index_names(name):
	return [name +"_"+ i  for i in indexes]
	
def two_index_names(name):
	return [name +"_"+ i + j for i in indexes for j in indexes if i != j]

def two_index_names_repeat(name):
    names = []
    for i in indexes:
        for j in indexes:
            if i != j:
                names.append(name + "_" + i + j)
            else:
                for k in indexes:
                    if k != i:
                        T = [i,k]
                        T.sort()
                        names.append(name + "_" + i + i + "_" + "".join(T))
    return names
    #return [name +"_"+ i + j for i in indexes for j in indexes]

def three_index_names(name):
	return [name +"_"+ i + j + k for i in indexes for j in indexes for k in indexes if i != j and i != k and j !=k]

def jjpS_names(name):
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for jj in Subsets(indexes,2):
            jjs = list(jj)
            jjs.sort()
            j = jjs[0]
            jp = jjs[1]
            
            if j not in S and jp not in S:
                names.append(name + "_" + j + jp + "_" + "".join(Ss))
                names.append(name + "_" + jp + j + "_" + "".join(Ss))

    return names

def ijS_names(name):
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for i in Ss:
            for j in indexes:
                if j not in Ss:
                    names.append(name + "_" + i + j + "_" + "".join(Ss))

    return names

def iS_names(name):
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for i in Ss:
            names.append(name + "_" + i + "_" + "".join(Ss))
    return names

def inotinS_names(name):
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for i in indexes:
            if i not in Ss:
                names.append(name + "_" + i + "_" + "".join(Ss))
    return names

def ijlS_names(name):
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for i in S:
            for l in S:
                for j in indexes:
                    if not j in S:
                        names.append(name + "_"+ i + j +l + "_" + "".join(Ss))
    return names
                    
def hS_names():
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for j in indexes:
            if not j in S:
                names.append("h_" + "".join(Ss) + "_" + j) 
    return names
        
    
Uring = PolynomialRing(QQ, two_index_names("al") + two_index_names_repeat("ga") +  two_index_names("a") + two_index_names("d") + two_index_names("t") + two_index_names("v") + two_index_names("r") + two_index_names_repeat("del") + two_index_names("b") + two_index_names("be") + two_index_names_repeat("ep") + two_index_names("psi")  + two_index_names("u") + two_index_names("g") + iS_names("pi") + two_index_names("k") + iS_names("s") 
                      + jjpS_names("c") + jjpS_names("cc") + ijS_names("a") + ijS_names("ac") + ijS_names("alin") + ijS_names("asq") + jjpS_names("hshsc") + ijS_names("b") +ijlS_names("d") + ijS_names("fhsc") + ijS_names("e") + ijS_names("r") + ijlS_names("s") +ijS_names("hhsc")
                      + two_index_names_repeat("theta") + two_index_names_repeat("pi") + inotinS_names("x") + ["aaa","bbb","ccc","ddd","eee"])

Rring = PolynomialRing(Uring, iS_names("f") + iS_names("h")+ hS_names())
    
Udict = Uring.gens_dict()
Rdict = Rring.gens_dict()
ExpandRing.<t> = LaurentSeriesRing(Uring)

def v(letter, a, b = "", c = "", d = None):
    #if g==2 and letter in ["c","e","ell"] and not type(c) == list:
    #    return 0
    
    #Some normalization
    if letter == "d" and type(d) == list:
        if a == d[0] and c == d[0]:
            return 0
    
    if letter == "h" and type(a) == list:
        return Rdict["h_" + "".join(a) + "_" + b]
    if type(c) == list:
        return Udict[letter + "_" + a + b + "_" + "".join(c)]
    if type(b) == list:
        try:
            return Udict[letter + "_" + a + "_" + "".join(b)]
        except:
            return Rdict[letter + "_" + a + "_" + "".join(b)]
    
    if not d is None:
        return Udict[letter + "_" + a + b + c + "_" + "".join(d)]
    try:
        return Udict[letter +"_"+ a + b +c]
    except KeyError:
        return Rdict[letter +"_"+ a + b +c]

We now create the relations.

In [4]:
def make_rel(lh, i, j, S = None):
    """
    lh is one of "ff", "fh", "hh", or "h2"
    i and j are the indexes appearing in the left hand side
    S is the set of indexes with size equal to g, must contain i,j, defaults to [i,j]

    j must be None in the case of "h2".
    """
    if S is None:
        if j is not None and g==2:
            S = [i,j]
        else:
            raise Exception("Not able to automatically find S.")

    S.sort()
    
    if not j is None:
        notij = copy(S)
        notij.remove(i)
        notij.remove(j)

    if lh == "ff":
        return ( v("f",i,S)*v("f",j,S) , v("al",j,i)*v("h",i,S) + v("al",i,j)*v("h",j,S) + v("ga",j,i)*v("f",i,S) + v("ga",i,j)*v("f",j,S) + sum((v("c",i,j,k)*v("f",k,S) for k in notij)) + v("a",i,j) )

    elif lh == "fh":
        return (v("f",i,S)*v("h",j,S) , v("d",i,j)*v("f",j,S)**2 + v("t",j,i)*v("h",i,S) + v("v",i,j)*v("h",j,S) + v("r",j,i)*v("f",i,S) + v("del",i,j)*v("f",j,S) + sum(( v("e", i,j,k)*v("f",k,S) for k in notij)) + v("b",i,j))

    elif lh == "hh":
        return (v("h",i,S)*v("h",j,S) , v("be",j,i)*v("f",i,S)**2 + v("be",i,j)*v("f",j,S)**2 + v("ep",j,i)*v("h",i,S) + v("ep",i,j)*v("h",j,S) + v("psi",j,i)*v("f",i,S) + v("psi",i,j)*v("f",j,S) + sum((v("ell",i,j,k)*v("f",k,S) for k in notij)) + v("u",i,j))

    elif lh == "h2":
        if not j is None:
            raise Exception("j must be None")

        noti = copy(S)
        noti.remove(i)

        return (v("h",i,S)**2, v("f",i,S)**3 + sum(( v("g",i,k)*v("h",k,S) for k in noti)) + v("pi",i,S)*v("f",i,S) + sum((v("k",i,k)*v("f",k,S) for k in noti)) + v("s",i,S))


First, we want to check the following.
### Lemma
The ring $\mathcal U^{ns}_{2,n}(ij)$ is generated by ten variables $\alpha_{ij}$, $\beta_{ij}$, $\gamma_{ij}$, $\epsilon_{ij}$, $\pi_i$.

We know that the ring is generated by all the coefficients appearing in the equations on page 9 of "... as moduli of $A_\infty$-structres". There are 23 letters appearing there. 3 of them ($c$, $e$, and $l$) do not occur in genus 2, and $q$, $r$, and $u$ (with one subscript), we make 0 by adding constants to $f$ and $g$. Hence, we need to eliminate 12 letters. 

By symmetry we will just check the case $i,j = 1,2$. We use the Buchberger algorithm to find relations.

In [79]:
lms = []
#the leading monomials
S = ["1","2"]
print "Warning: S is hard coded here!!!!"

for i, j in Combinations(S,2):
    lms.append( (v("f",i,S)*v("f",j,S), make_rel , ("ff", i, j,S)))
    lms.append( (v("h",i,S)*v("h",j,S), make_rel, ("hh", i, j,S)))

for i in S:
    lms.append( (v("h",i,S)^2, make_rel , ("h2", i, None, S)))  
    for j in S:
        if j==i:
            continue
        lms.append( (v("f",i,S)*v("h",j,S), make_rel,("fh", i, j,S)))

def doS(r1,r2):
    """
    Implements the Buchberger algorithm to find relations between the coefficients.
    """
    L = lcm(r1[0].change_ring(QQ), r2[0].change_ring(QQ))
    factor1 = L.quo_rem(r1[0])[0]
    factor2 = L.quo_rem(r2[0])[0]
    S = factor1 * r1[1] - factor2 * r2[1]
    newS = reduce_lm(S)

    #print "Relations Found:"
    for c in newS.coefficients():
        yield c
        
def reduce_lm(fff):
    """
    Outsource this code from doS.
    """
    fff1 = fff
    done = False
    
    while not done:
        new_fff = 0
        for m in fff1.monomials():
            can_be_reduced = False
            for lm in lms:
                if lm[0].divides(m):
                    can_be_reduced = True
                    #newS += make_rel(lm[1], lm[2], lm[3])[1] *(m/lm[0]).numerator()*S1.monomial_coefficient(m)
                    new_fff += (lm[1](*lm[2]))[1] * (m/lm[0]).numerator()*fff1.monomial_coefficient(m)
                    break
            if not can_be_reduced:
                new_fff += m*fff1.monomial_coefficient(m)


        if fff1 == new_fff:
            done = True
        else:
            #print "need to go again"
            fff1 = new_fff
    return new_fff

monos_to_save = ["al_12","al_21", "be_12", "be_21", "ga_12", "ga_21", "pi_1", "pi_2", "ep_12", "ep_21"]
def eliminate_vars(r1,r2):
    for rrr in doS(r1,r2):
        do_monos =[]
        for m in rrr.monomials():
            if m.degree() == 1:
                do_monos.append(m)

        doe_monos = []
        for m in do_monos:
            if str(m) not in monos_to_save:
                doe_monos.append(m)

        if len(doe_monos) == 0:
            if len(do_monos) >0:
                raise Exception("Must eliminate reserved var!!!!!")
            #otherwise do nothing, can't eliminate a ver
        else:

            m = doe_monos[0]

            #rels_found[m] = m-1/rrr.monomial_coefficient(m)*rrr
            print "Adding relations of the form:",m, "=",m-1/rrr.monomial_coefficient(m)*rrr
            for a,b in make_all_subs([str(m), str(m-1/rrr.monomial_coefficient(m)*rrr)]): 
                #Don't love this method of making sure we don't double eliminate a relation...
                if len(str(m)) == len(str(eval(a))):
                    Udict[str(eval(a))] = eval(b)
            
            reduceUdict()

            
def reduceUdict():
    
    vUdict = {Uring(k):val for k,val in Udict.items()}
    for k, val in Udict.items(): 
        Udict[k] = val.subs(vUdict)
        
import re
def make_all_subs(strs):
    indexes_involved = set()
    for st in strs:
        for match in re.finditer(r"[a-z]+_(\d+)(?:_(\d+))?",st):
            indexes_involved.update(match.group(1))
            if not match.group(2) is None:
                indexes_involved.update(match.group(2))

    indexes_involved = list(indexes_involved)

    for s in Subsets(indexes, len(indexes_involved)):
        for p in Permutations(s):

            def permute_indices(matchobj):
                #Tracer()()
                new_indexes = []
                for i in matchobj.group(2):
                    new_indexes += p[indexes_involved.index(i)]
                if matchobj.group(3) is None:
                    return "v('" + matchobj.group(1) + "', '" +  "','".join(new_indexes) + "')"
                #else
                new_indexes2 = []
                for i in matchobj.group(3):
                    new_indexes2 += p[indexes_involved.index(i)]
                    
                if matchobj.group(1) == "h":
                    #it is an S, j type
                    new_indexes.sort()
                    return "v('" + matchobj.group(1) + "', " + str(new_indexes) + "', '" + ", '".join(new_indexes2) + "')"
                #else it is a i, S type
                new_indexes2.sort()
                return "v('" + matchobj.group(1) + "', '" +  "','".join(new_indexes) + "', " + str(new_indexes2) + ")"
                

            yield tuple((re.sub(r"([a-z]+)_(\d+)(?:_(\d+))?", permute_indices, str).replace("^","**") for str in strs))
            #yield tuple((re.sub(r"([a-z]+)_([0-9])([0-9])?([0-9])?", permute_indices, str).replace("^","**") for str in strs))

Doing Buchberger on the $f_1f_2$ and $f_1h_2$ equations elimnates $d_{ij}$, $\delta_{12}$, 3 more.

In [80]:
eliminate_vars(make_rel("ff","1","2"),make_rel("fh","1","2"))

Adding relations of the form: d_12 = al_12
Adding relations of the form: del_12 = ga_21*d_12 + al_21*be_12
Adding relations of the form: v_12 = ga_12
Adding relations of the form: b_12 = -t_21*r_12 - ga_12*r_21 + ga_21*del_12 + al_21*psi_12 + al_12*pi_2_12
Adding relations of the form: a_12 = t_12*t_21 - ga_21*v_12 + al_12*r_21 - al_21*ep_12


Find 4 more

In [7]:
eliminate_vars(make_rel("hh","1","2"),make_rel("fh","1","2"))

Adding relations of the form: t_21 = be_21
Adding relations of the form: psi_21 = 3*al_12*al_21*ga_21 + ga_12*be_21
Adding relations of the form: r_21 = -al_12*al_21^2 + ep_21
Adding relations of the form: u_12 = 3*al_12*al_21*ga_12*ga_21 + al_12*al_21*t_12*t_21 + 2*al_12^2*al_21*r_21 + 3*al_12^2*ga_21*be_21 - al_12*al_21^2*ep_12 - ga_21^2*be_12 - r_21*ep_12 - ga_21*psi_12 + ga_12*psi_21 + t_21*pi_1_12


And here are the last 3:

In [8]:
eliminate_vars(make_rel("hh","1","2"),make_rel("h2","1",None,["1","2"]))

Adding relations of the form: g_12 = -al_12^3
Adding relations of the form: k_12 = -3*al_12^2*ga_12 + be_12^2
Adding relations of the form: s_1_12 = 2*al_12^4*al_21^2 - 6*al_12^2*ga_12*ga_21 - al_12^2*be_12*be_21 - 2*al_12^3*ep_21 - al_12*al_21^2*g_12 - ga_12^3 + 2*ga_21*be_12^2 + ep_12^2 + ep_21*g_12 - ga_12*pi_1_12


Maybe you are worried about cyclic dependencies, but check out the `Udict` to make sure all is legit:

In [9]:
for k, val in Udict.items():
    print "~~~", k, "=", val

~~~ cc_12_34 = cc_12_34
~~~ x_4_12 = x_4_12
~~~ r_42_34 = r_42_34
~~~ hshsc_23_14 = hshsc_23_14
~~~ ac_43_14 = ac_43_14
~~~ theta_33_23 = theta_33_23
~~~ u_14 = -2*al_14^3*al_41^3 + 3*al_14*al_41*ga_14*ga_41 + 3*al_14^2*ga_41*be_41 + al_14*al_41*be_14*be_41 + 2*al_14^2*al_41*ep_41 - 2*ga_41^2*be_14 + ga_14^2*be_41 - ep_14*ep_41 + be_41*pi_1_14
~~~ u_13 = -2*al_13^3*al_31^3 + 3*al_13*al_31*ga_13*ga_31 + 3*al_13^2*ga_31*be_31 + al_13*al_31*be_13*be_31 + 2*al_13^2*al_31*ep_31 - 2*ga_31^2*be_13 + ga_13^2*be_31 - ep_13*ep_31 + be_31*pi_1_13
~~~ u_12 = -2*al_12^3*al_21^3 + 3*al_12*al_21*ga_12*ga_21 + 3*al_12^2*ga_21*be_21 + al_12*al_21*be_12*be_21 + 2*al_12^2*al_21*ep_21 - 2*ga_21^2*be_12 + ga_12^2*be_21 - ep_12*ep_21 + be_21*pi_1_12
~~~ c_23_14 = c_23_14
~~~ s_234_24 = s_234_24
~~~ ac_24_23 = ac_24_23
~~~ pi_44_24 = pi_44_24
~~~ b_23_24 = b_23_24
~~~ d_234_24 = d_234_24
~~~ s_313_34 = s_313_34
~~~ fhsc_31_34 = fhsc_31_34
~~~ d_324_34 = d_324_34
~~~ ac_32_13 = ac_32_13
~~~ pi_33_13 = pi_33_1

Now, let us see what other relations we have.

In [10]:
for r in doS(make_rel("ff","1","2"),make_rel("fh","1","2")):
    print factor(r)
    print

(4) * al_12 * al_21^2 * (-al_21*ep_12 + al_12*ep_21)



In [11]:
for r in doS(make_rel("hh","1","2"),make_rel("fh","1","2")):
    print factor(r)
    print

(-1) * al_12 * (3*al_12^2*al_21*ga_21 + 3*al_12*al_21^2*be_12 + 3*al_21*ga_12^2 - 2*be_12*ep_21 + al_21*pi_1_12)

(-1) * (3*al_12*al_21^2*ga_12 + 3*al_12^2*al_21*be_21 + 3*al_12*ga_21^2 - 2*be_21*ep_12 + al_12*pi_2_12)

-3*al_12^3*al_21^3*ga_12 - 3*al_12^4*al_21^2*be_21 - 9*al_12^3*al_21*ga_21^2 - 6*al_12^2*al_21^2*ga_21*be_12 - 6*al_12*al_21*ga_12^2*ga_21 + 3*al_12*al_21^2*ga_12*ep_12 + 3*al_12^2*al_21*be_21*ep_12 + 2*al_12^3*be_21*ep_21 - al_12^3*al_21*pi_2_12 + 3*al_12*ga_21^2*ep_12 + 4*al_12*ga_21*be_12*ep_21 - 2*al_12*al_21*ga_21*pi_1_12 - 2*be_21*ep_12^2 + al_12*ep_12*pi_2_12



In [12]:
for r in doS(make_rel("hh","1","2"),make_rel("h2","1",None,["1","2"])):
    print factor(r)
    print

(-1) * (3*al_12*al_21^2*ga_12 + 3*al_12^2*al_21*be_21 + 3*al_12*ga_21^2 - 2*be_21*ep_12 + al_12*pi_2_12)

(-1) * (3*al_12^3*ga_21 + 3*al_12^2*al_21*be_12 + 3*al_12*ga_12^2 - 2*be_12*ep_12 + al_12*pi_1_12)

(-1) * (3*al_12*al_21^2*ga_12^2 + 3*al_12^2*al_21*ga_12*be_21 + 3*al_12*ga_12*ga_21^2 - 6*al_12*al_21*ga_21*ep_12 + 6*al_12^2*ga_21*ep_21 - 2*ga_12*be_21*ep_12 + al_12*ga_12*pi_2_12)

(-1) * (3*al_12^3*ga_21^2 + 3*al_12^2*al_21*ga_21*be_12 + 3*al_12*ga_12^2*ga_21 - 6*al_12*al_21*ga_12*ep_12 + 6*al_12^2*ga_12*ep_21 - 2*ga_21*be_12*ep_12 + al_12*ga_21*pi_1_12)

(-2) * al_21 * al_12 * (-al_21*ep_12 + al_12*ep_21)

(-1) * (9*al_12^3*al_21^2*ga_12*ga_21 + 9*al_12^4*al_21*ga_21*be_21 + 4*al_12^3*al_21^3*ep_12 - 4*al_12^4*al_21^2*ep_21 + 3*al_12*al_21^2*ga_12^3 + 3*al_12^3*ga_21^3 - 6*al_12^2*al_21*ga_21^2*be_12 + 3*al_12^2*al_21*ga_12^2*be_21 - 3*al_12*ga_12^2*ga_21^2 - 6*al_12*al_21*ga_12*ga_21*ep_12 - 6*al_12^2*ga_21*be_21*ep_12 - 2*al_12*al_21*be_12*be_21*ep_12 + 2*al_12*al_21^2*ep_12^2

In [13]:
str_relations = (list(make_all_subs(["al_21 *( 3*al_12^2*ga_21 + 3*al_12*al_21*be_12 + 3*ga_12^2 + pi_1_12) - 2*be_12*ep_21"])) +
                 list(make_all_subs(["al_12*(3*al_12^2*ga_21 + 3*al_12*al_21*be_12 + 3*ga_12^2 + pi_1_12 )- 2*be_12*ep_12"])))

relations_in_A10 = [eval(r[0]) for r in str_relations]
relations_in_A10 += [Uring("al_12*ep_21 - al_21*ep_12"),
                  Uring("be_21*(3*al_12^2*ga_21 + 3*al_12*al_21*be_12 + 3*ga_12^2 + pi_1_12) - be_12*( 3*al_21^2*ga_12 + 3*al_21*al_12*be_21 + 3*ga_21^2 + pi_2_12)")]

Uring_quo = Uring.quotient(relations_in_A10)

This shows that the $\tilde{\pi_i}$ for this normalization differs from the "all zero" normalization by replacing the 3/2 coefficient with 3.

In [14]:
for r in doS(make_rel("ff","1","2"),make_rel("fh","1","2")):
    print Uring_quo(r)

0


In [15]:
for r in doS(make_rel("hh","1","2"),make_rel("fh","1","2")):
    print Uring_quo(r)

0
0
0


In [16]:
for r in doS(make_rel("hh","1","2"),make_rel("h2","1",None,["1","2"])):
    print Uring_quo(r)

0
0
0
0
0
0


In [17]:
for r in doS(make_rel("hh","1","2"),make_rel("fh","2","1")):
    print Uring_quo(r)

0
0
0
0


Now, let us check that the point that we computed in our paper lies on the equations.

In [18]:
c_point_dict ={
    v("al","1","2") :-1,
    v("al","2","1") :1,
    v("be","1","2") : 3/2,
    v("be","2","1") :3/2,
    v("ga","1","2") : 5/4,
    v("ga","2","1") :5/4,
    v("ep","1","2") :-5/4,
    v("ep","2","1") :5/4,
    v("pi","1",["1","2"]) : -3/16,
    v("pi","2",["1","2"]) : -3/16,
}

for r in relations_in_A10:
    print r
    print r.subs(c_point_dict)
    print "*  *  *"
#NOTE: This will have some things you don't want for n > 2 

3*al_12^2*al_21*ga_21 + 3*al_12*al_21^2*be_12 + 3*al_21*ga_12^2 - 2*be_12*ep_21 + al_21*pi_1_12
0
*  *  *
3*al_12*al_21^2*ga_12 + 3*al_12^2*al_21*be_21 + 3*al_12*ga_21^2 - 2*be_21*ep_12 + al_12*pi_2_12
0
*  *  *
3*al_13^2*al_31*ga_31 + 3*al_13*al_31^2*be_13 + 3*al_31*ga_13^2 - 2*be_13*ep_31 + al_31*pi_1_13
3*al_13^2*al_31*ga_31 + 3*al_13*al_31^2*be_13 + 3*al_31*ga_13^2 - 2*be_13*ep_31 + al_31*pi_1_13
*  *  *
3*al_13*al_31^2*ga_13 + 3*al_13^2*al_31*be_31 + 3*al_13*ga_31^2 - 2*be_31*ep_13 + al_13*pi_3_13
3*al_13*al_31^2*ga_13 + 3*al_13^2*al_31*be_31 + 3*al_13*ga_31^2 - 2*be_31*ep_13 + al_13*pi_3_13
*  *  *
3*al_14^2*al_41*ga_41 + 3*al_14*al_41^2*be_14 + 3*al_41*ga_14^2 - 2*be_14*ep_41 + al_41*pi_1_14
3*al_14^2*al_41*ga_41 + 3*al_14*al_41^2*be_14 + 3*al_41*ga_14^2 - 2*be_14*ep_41 + al_41*pi_1_14
*  *  *
3*al_14*al_41^2*ga_14 + 3*al_14^2*al_41*be_41 + 3*al_14*ga_41^2 - 2*be_41*ep_14 + al_14*pi_4_14
3*al_14*al_41^2*ga_14 + 3*al_14^2*al_41*be_41 + 3*al_14*ga_41^2 - 2*be_41*ep_14 + al_14*pi_4

Let us make sure that u is symmetric.

In [19]:
Uring_quo(Udict["u_21"]-Udict["u_12"])

0

In [20]:
(Udict["u_21"]+Udict["u_12"])/2

-2*al_12^3*al_21^3 + 3*al_12*al_21*ga_12*ga_21 + 3/2*al_21^2*ga_12*be_12 + 3/2*al_12^2*ga_21*be_21 + al_12*al_21*be_12*be_21 + al_12*al_21^2*ep_12 + al_12^2*al_21*ep_21 - 1/2*ga_21^2*be_12 - 1/2*ga_12^2*be_21 - ep_12*ep_21 + 1/2*be_21*pi_1_12 + 1/2*be_12*pi_2_12

In [21]:
print Udict["b_12"]
print
print Udict["b_12"].subs({v("pi","2",["1","2"]): Uring("-3*al_21^2*ga_12 - 3*al_21*al_12*be_21 - 3*ga_21^2 + pi_2_12") }) + Uring("2*be_21*ep_12- al_12*pi_2_12")

4*al_12*al_21^2*ga_12 + al_12^2*al_21*be_21 + al_12*ga_21^2 + 2*al_21*ga_21*be_12 - be_21*ep_12 - ga_12*ep_21 + al_12*pi_2_12

al_12*al_21^2*ga_12 - 2*al_12^2*al_21*be_21 - 2*al_12*ga_21^2 + 2*al_21*ga_21*be_12 + be_21*ep_12 - ga_12*ep_21


Now, let us find formulas for all the coefficients in the expansions.

In [133]:
expand_at_dict = dict()
for j in indexes:
    expand_at_dict[j] = dict()
    for T in Subsets(indexes,2):
        Ts = list(T)
        Ts.sort()
        
        for i in indexes:
            if i in Ts:
                f = v("f",i,Ts)
                h = v("h",i,Ts)
                if j in Ts:
                    if i != j:
                        expand_at_dict[j][f] = v("al",i,j)/t + v("ga",i,j) + v("del",i,j)*t + v("pi",i,j)*t^2 + O(t^3)
                        expand_at_dict[j][h] = v("be",i,j)/t + v("ep",i,j) + v("theta",i,j)*t + O(t^2)
                    else:
                        expand_at_dict[j][f] = 1/t^2 + v("ga",i,j,Ts) + v("del",i,j,Ts)*t + v("pi",i,j,Ts)*t^2 + O(t^3)
                        expand_at_dict[j][h] = 1/t^3 + v("ep",i,j,Ts) + v("theta",i,j,Ts)*t + O(t^2)
                else: #j is not in Ts
                    expand_at_dict[j][f] = v("b",i,j,Ts) + O(t)
                    expand_at_dict[j][h] = v("e",i,j,Ts) + O(t)
            else: #i is not in Ts
                hS = v("h",Ts,i)
                if j in Ts:
                    expand_at_dict[j][hS] = v("a",j,i,Ts)/t + v("ac",j,i,Ts) + v("alin",j,i,Ts)*t + v("asq", j,i,Ts)*t^2 + O(t^3) #check the index order one more time!!!

                        
                else: #j not in Ts
                    if i != j:
                        expand_at_dict[j][hS] = v("c",i,j,Ts) + v("cc",i,j,Ts)*t + O(t^2)
                    else:
                        expand_at_dict[j][hS] = 1/t + v("x",i,Ts) + O(t)
                        

                
def expand_at(func, point):
    return func.subs(expand_at_dict[point])


def find_rels_by_expansion(point_index, eqn):
    expansion = expand_at(eqn[0]-eqn[1], point_index)
    
    print "Some relations"
    for value in expansion.coefficients():
        print value

In [134]:
find_rels_by_expansion("1", make_rel("ff","1","2"))
find_rels_by_expansion("1", make_rel("fh","1","2"))
find_rels_by_expansion("2", make_rel("fh","1","2"))
find_rels_by_expansion("1", make_rel("hh","1","2"))
find_rels_by_expansion("1", make_rel("h2","1",None,["1","2"]))
find_rels_by_expansion("2", make_rel("h2","1",None,["1","2"]))
find_rels_by_expansion("3", make_rel("ff","1","2"))
#expand_at("3", make_rel("ff","1","2")) Don't do this, we are in genus 2 for now.
#TODO: put these relations in the Udict???

Some relations
al_21*ga_11_12 - al_21*ga_12 - al_12*be_21 + del_21
-ga_12*ga_21 + al_21*del_11_12 - al_21*ep_11_12 - al_12*ep_21 - a_12 + pi_21
Some relations
-t_21 + be_21
-al_21^2*d_12 - r_21 + ep_21
-2*al_21*ga_21*d_12 - al_21*del_12 + ga_11_12*be_21 - v_12*be_21 + theta_21
Some relations
al_12 - d_12
ga_12 - v_12
-2*ga_22_12*d_12
-al_12*r_21 - 2*d_12*del_22_12 - t_21*be_12 + al_12*ep_22_12 + pi_12
Some relations
-al_21^2*be_12 - 2*ga_11_12*be_21 - psi_21 + theta_21
Some relations
-3*ga_11_12
-3*del_11_12 + 2*ep_11_12
-3*ga_11_12^2 - pi_1_12 + 2*theta_11_12 - 3*pi_11_12
Some relations
-al_12^3 - g_12
-3*al_12^2*ga_12 + be_12^2 - k_12
-3*al_12*ga_12^2 - 3*al_12^2*del_12 + 2*be_12*ep_12 - al_12*pi_1_12
-ga_12^3 - 6*al_12*ga_12*del_12 - 3*al_12^2*pi_12 + ep_12^2 - ep_22_12*g_12 - ga_12*pi_1_12 - ga_22_12*k_12 + 2*be_12*theta_12 - s_1_12
Some relations
-ga_21*b_13_12 - ga_12*b_23_12 + b_13_12*b_23_12 - al_21*e_13_12 - al_12*e_23_12 - a_12


In [135]:
def hShS(S, j, jp): #not tested, not needed for g=2
    return (v("h", S, j)*v("h",S,jp), v("c", jp, j, S)*v("h",S,j) +  v("c", j, jp, S)*v("h",S,jp) + sum((v("a", i,j,S)*v("a",i,jp,S)*v("f",i,S) for i in S)) +  v("hshsc",jp,j,S))
            
def fhS(S, i,j):
    if not i in S:
        raise Excpetion("i should be in S")
    return (v("f",i,S)*v("h", S, j), v("b",i,j,S)*v("h",S,j) + v("a",i,j,S)*v("h",i,S) + sum((v("d",i,j,l,S)*v("f",l,S) for l in S)) + v("fhsc",i,j,S))
            
def hhS(S,i,j):
    if not i in S:
        raise Excpetion("i should be in S")
    return (v("h",i,S)*v("h",S,j), v("e",i,j,S)*v("h",S,j) +  v("a",i,j,S)*v("f",i,S)**2 + v("r",i,j,S)*v("h",i,S) + sum((v("s",i,j,l,S)*v("f",l,S) for l in S)) + v("hhsc",i,j,S))


In [136]:
for S in Subsets(indexes,g):
    Ss = list(S)
    Ss.sort()
    for j in indexes:
        if j in S:
            continue
        for i in S:
            lms.append((v("f",i,Ss)*v("h",Ss,j,Ss), fhS, (Ss,i,j)))
            lms.append((v("h",i,Ss)*v("h",Ss,j,Ss), hhS, (Ss,i,j)))
            
        for jp in indexes:
            if jp in S or jp == j:
                continue
            lms.append((v("h",Ss,j)*v("h",Ss,jp), hShS, (Ss,j,jp)))


In [137]:
find_rels_by_expansion("1", hhS(["1","2"],"2","3"))
find_rels_by_expansion("2", hhS(["1","2"],"2","3"))
find_rels_by_expansion("3", hhS(["1","2"],"2","3"))

find_rels_by_expansion("1", fhS(["1","2"],"2","3"))
find_rels_by_expansion("2", fhS(["1","2"],"2","3"))
find_rels_by_expansion("3", fhS(["1","2"],"2","3"))

Some relations
-al_21^2*a_23_12 + be_21*a_13_12 - s_231_12
-2*al_21*ga_21*a_23_12 + ep_21*a_13_12 + be_21*ac_13_12 - a_13_12*e_23_12 - be_21*r_23_12 - al_21*s_232_12
-ga_21^2*a_23_12 - 2*al_21*del_21*a_23_12 + ep_21*ac_13_12 + be_21*alin_13_12 - ac_13_12*e_23_12 - ep_21*r_23_12 - ga_11_12*s_231_12 - ga_21*s_232_12 + a_13_12*theta_21 - hhsc_23_12
Some relations
ac_23_12 - r_23_12
-2*ga_22_12*a_23_12 + alin_23_12 - s_232_12
-2*del_22_12*a_23_12 + ep_22_12*a_23_12 - a_23_12*e_23_12 - al_12*s_231_12 + asq_23_12
Some relations
Some relations
al_21*a_13_12 - d_231_12
ga_21*a_13_12 - be_21*a_23_12 + al_21*ac_13_12 - a_13_12*b_23_12 - al_21*d_232_12
del_21*a_13_12 - ep_21*a_23_12 + ga_21*ac_13_12 + al_21*alin_13_12 - ac_13_12*b_23_12 - ga_11_12*d_231_12 - ga_21*d_232_12 - fhsc_23_12
del_21*ac_13_12 + ga_21*alin_13_12 + al_21*asq_13_12 - alin_13_12*b_23_12 - del_11_12*d_231_12 - del_21*d_232_12 - a_23_12*theta_21 + a_13_12*pi_21
Some relations
ac_23_12 - d_232_12
ga_22_12*a_23_12 - a_23_12*b_23

In [138]:
def transition(H0gens, f, coefs_to_check):
    """
    This function will find the coefficients of the expression of f in the basis given H0gens. 
    coefs_to_check is a list of pairs (point, exponent) of terms in the expansion to use to 
    determine the coefficients.
    
    Note: The user is responsible for making sure that the parameters are compatible!!
    """
    H0gens = [Rring(g) for g in H0gens]
    f = Rring(f)
    
    M = Matrix(Uring, len(coefs_to_check),len(H0gens),)
    b = vector(Uring,len(coefs_to_check))

    for i, (point, expon) in enumerate(coefs_to_check):
        b[i] = expand_at(f,point)[expon]
        for j,gen in enumerate(H0gens):
            M[i,j] = expand_at(gen,point)[expon]
            
    result = M.augment(b).echelon_form()
    for row in result.rows():
        print row
        
    return result

First, let's compute $f_{1,12}$ restricted to $\tilde{\mathcal U}(13)$. The space $H^0(2p_1 + p_2 + p_3)$ is spanned by $1$, $f_{1,13}$, and $h_2(13)$. We are going to skip computing the constant for now.

In [145]:
M=transition(["f_1_12","h_12_3","1"],
          "f_1_13",
          [("1",-2),("2",-1),("3",-1),("2",0)]);
f_1_13_12 = Rring("f_1_12 + al_13*h_12_3 -al_13*ac_23_12 - ga_12 + b_12_13")

(1, 0, 0, 1)
(0, 1, 0, al_13)
(0, 0, 1, -al_13*ac_23_12 - ga_12 + b_12_13)
(0, 0, 0, -al_13*a_23_12 - al_12)


In [147]:
M=transition(["h_1_12","f_1_12","h_12_3","1"],
            "h_1_13",
            [("1",-3),("3",-1),("2",-1),("2",0)]);
h_1_13_12 = Rring("h_1_12 + bbb*f_1_12 + be_13*h_12_3 + ccc")

(1, 0, 0, 0, 1)
(0, 0, 1, 0, be_13)
(0, ga_12, 0, 1, -be_13*ac_23_12 - ep_12 + e_12_13)
(0, al_12, 0, 0, -be_13*a_23_12 - be_12)


In [148]:
#r=make_rel("h2","1",None,["1","3"])

ff=h_1_13_12^2 - f_1_13_12^3 #-  Rring("pi_1_13*f_1_13 + k_13*f_3_13 + g_13*h_3_13 + s_1_13")

print ff

expand_at(ff,"1")

-f_1_12^3 + (-3*al_13)*f_1_12^2*h_12_3 + (-3*al_13^2)*f_1_12*h_12_3^2 + (-al_13^3)*h_12_3^3 + (3*al_13*ac_23_12 + bbb^2 + 3*ga_12 - 3*b_12_13)*f_1_12^2 + 2*bbb*f_1_12*h_1_12 + h_1_12^2 + (6*al_13^2*ac_23_12 + 6*al_13*ga_12 - 6*al_13*b_12_13 + 2*be_13*bbb)*f_1_12*h_12_3 + 2*be_13*h_1_12*h_12_3 + (3*al_13^3*ac_23_12 + 3*al_13^2*ga_12 - 3*al_13^2*b_12_13 + be_13^2)*h_12_3^2 + (-3*al_13^2*ac_23_12^2 - 6*al_13*ga_12*ac_23_12 + 6*al_13*ac_23_12*b_12_13 - 3*ga_12^2 + 6*ga_12*b_12_13 - 3*b_12_13^2 + 2*bbb*ccc)*f_1_12 + 2*ccc*h_1_12 + (-3*al_13^3*ac_23_12^2 - 6*al_13^2*ga_12*ac_23_12 + 6*al_13^2*ac_23_12*b_12_13 - 3*al_13*ga_12^2 + 6*al_13*ga_12*b_12_13 - 3*al_13*b_12_13^2 + 2*be_13*ccc)*h_12_3 + al_13^3*ac_23_12^3 + 3*al_13^2*ga_12*ac_23_12^2 - 3*al_13^2*ac_23_12^2*b_12_13 + 3*al_13*ga_12^2*ac_23_12 - 6*al_13*ga_12*ac_23_12*b_12_13 + 3*al_13*ac_23_12*b_12_13^2 + ga_12^3 - 3*ga_12^2*b_12_13 + 3*ga_12*b_12_13^2 - b_12_13^3 + ccc^2


(-3*al_13*a_13_12 + 2*bbb)*t^-5 + (-3*al_13^2*a_13_12^2 + 2*be_13*a_13_12 - 3*al_13*ac_13_12 + 3*al_13*ac_23_12 + bbb^2 - 3*ga_11_12 + 3*ga_12 - 3*b_12_13)*t^-4 + (-al_13^3*a_13_12^3 - 6*al_13^2*a_13_12*ac_13_12 + 6*al_13^2*a_13_12*ac_23_12 - 6*al_13*ga_11_12*a_13_12 + 6*al_13*ga_12*a_13_12 - 6*al_13*a_13_12*b_12_13 + 2*be_13*a_13_12*bbb + 2*be_13*ac_13_12 - 3*al_13*alin_13_12 + 2*ga_11_12*bbb - 3*del_11_12 + 2*ep_11_12 + 2*ccc)*t^-3 + (-3*al_13^3*a_13_12^2*ac_13_12 + 3*al_13^3*a_13_12^2*ac_23_12 - 3*al_13^2*ga_11_12*a_13_12^2 + 3*al_13^2*ga_12*a_13_12^2 - 3*al_13^2*a_13_12^2*b_12_13 + be_13^2*a_13_12^2 - 3*al_13^2*ac_13_12^2 + 6*al_13^2*ac_13_12*ac_23_12 - 3*al_13^2*ac_23_12^2 - 6*al_13^2*a_13_12*alin_13_12 - 6*al_13*del_11_12*a_13_12 - 6*al_13*ga_11_12*ac_13_12 + 6*al_13*ga_12*ac_13_12 + 6*al_13*ga_11_12*ac_23_12 - 6*al_13*ga_12*ac_23_12 - 6*al_13*ac_13_12*b_12_13 + 6*al_13*ac_23_12*b_12_13 + 2*be_13*ac_13_12*bbb + 2*ga_11_12*bbb^2 - 3*ga_11_12^2 + 6*ga_11_12*ga_12 - 3*ga_12^2 + 2*be

In [142]:
expand_at(Rring("f_1_12^2*h_12_3"),"1")

a_13_12*t^-5 + ac_13_12*t^-4 + (2*ga_11_12*a_13_12 + alin_13_12)*t^-3 + (2*del_11_12*a_13_12 + 2*ga_11_12*ac_13_12 + asq_13_12)*t^-2 + O(t^-1)

In [104]:
reduce_lm(ff)

(-3*al_13)*f_1_12^2*h_12_3 + (-3*al_13^2)*f_1_12*h_12_3^2 + (-al_13^3)*h_12_3^3 + bbb^2*f_1_12^2 + 2*bbb*f_1_12*h_1_12 + 2*be_13*bbb*f_1_12*h_12_3 + 2*be_13*h_1_12*h_12_3 + be_13^2*h_12_3^2 + pi_1_12*f_1_12 + k_12*f_2_12 + (-pi_1_13)*f_1_13 + (-k_13)*f_3_13 + g_12*h_2_12 + (-g_13)*h_3_13 + s_1_12 - s_1_13

Now, let us compute $f_{1,12}$ restricted to $\tilde{\mathcal U}(23)$. Again, we may omit the constants.

In [40]:
M=transition(["f_2_23","f_3_23", "h_23_1", "h_23_1^2"],
          "f_1_12",
          [("1",-2),("2",-2),("2",-1),("3",-2),("3",-1)]);

(1, 0, 0, 0, -a_21_23^2)
(0, 1, 0, 0, -a_31_23^2)
(0, 0, 0, 1, 1)
(0, 0, a_21_23, 0, al_32*a_31_23^2 - 2*a_21_23*ac_21_23 + al_12)
(0, 0, a_31_23, 0, al_23*a_21_23^2 - 2*a_31_23*ac_31_23)


In [43]:
#Note that a_21_23/a_31_23 = a_23_12
#This is the formula for al_12!!!!!!
M[4,4]*Uring("-a_23_12") + M[3,4]

-al_23*a_23_12*a_21_23^2 + al_32*a_31_23^2 + 2*a_23_12*a_31_23*ac_31_23 - 2*a_21_23*ac_21_23 + al_12

From the equation above, we see that 

$\alpha_{12}x_{12}= \alpha_{23} x_{31} a_{21}(23)^2 - \alpha_{32} a_{31}(23)^2 x_{12} - 2 x_{31} a_{31}(23) \tilde{a}_{31}(23)$$ + 2 a_{21}(23) \tilde{a}_{21}(23)x_{12}$

In [47]:
M=transition(["h_34_1","h_34_1^2","h_34_2", "f_3_34", "f_4_34"],
          "f_1_12",
          [("1",-2),("2",-1),("3",-2),("3",-1),("4",-2),("4",-1)])

(0, 1, 0, 0, 0, 1)
(0, 0, 1, 0, 0, al_12)
(0, 0, 0, 1, 0, -a_31_34^2)
(0, 0, 0, 0, 1, -a_41_34^2)
(a_31_34, 0, 0, 0, 0, al_43*a_41_34^2 - al_12*a_32_34 - 2*a_31_34*ac_31_34)
(a_41_34, 0, 0, 0, 0, al_34*a_31_34^2 - al_12*a_42_34 - 2*a_41_34*ac_41_34)


In [50]:
M[4,5]*Rring("a_41_34") - M[5,5]*Rring("a_31_34")

-al_34*a_31_34^3 + al_43*a_41_34^3 - al_12*a_32_34*a_41_34 + al_12*a_31_34*a_42_34 - 2*a_31_34*a_41_34*ac_31_34 + 2*a_31_34*a_41_34*ac_41_34

Apply a Plucker relation to the coefficient of $\alpha_{12}$ to turn it into $\pm x_{12}/x_{34}$. Multiply by $x_{34}$ and we should get our formula for $\alpha_{12}x_{12}$ on $\mathcal{U}(34)$.

So we get
$\alpha_{12}x_{12} = \pm x_{34}(-\alpha_{34}a_{31}(34)^3 + \alpha_{43}a_{41}(34)^3  - 2 a_{31}(34)a_{41}(34)\tilde{a}_{31}(34) + 2a_{31}(34)a_{41}(34)\tilde{a}_{41}(34))$

Question to self: Do we need to check that the formulas on $\mathcal{U}(23)$ and $\mathcal{U}(34)$ are compatible on the intersection?

Answer: Yes, compatibility is relevant. But let's shelve this for now because we think $\alpha_{ij}$ is going to be in an integral dependence, not be a generator.